In [1]:
import pandas as pd
import numpy as np
import tqdm
import pickle
from datetime import datetime, timedelta
import os

data_dir = './data/'
file = 'userid-timestamp-artid-artname-traid-traname.tsv'

In [2]:
df = pd.read_csv(data_dir + file, delimiter="\t", header=None, error_bad_lines=False)
df.describe()
drop_na_df = df.dropna(axis=0, how='any')
#drop_na_df.head(5)
drop_na_df.describe()

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


,0,1,2,3,4,5
count,16936134,16936134,16936134,16936134,16936134,16936134
unique,992,15631466,83905,81751,960402,693231
top,user_000949,2009-02-26T21:29:15Z,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,db16d0b3-b8ce-4aa8-a11a-e4d53cc7f8a6,Intro
freq,172042,193,111488,111488,3991,14908


In [2]:
file_name = 'original_set'
#drop_na_df.to_pickle(data_dir + file_name)
drop_na_df = pd.read_pickle(data_dir + file_name)

In [3]:
most_close_time = datetime.strptime("2009-05-05 23:59:59", "%Y-%m-%d %H:%M:%S")
print(most_close_time)
most_early_time = most_close_time - timedelta(days=30)
print(most_early_time)

2009-05-05 23:59:59
2009-02-04 23:59:59


In [4]:
temp_df = drop_na_df.copy()
f = lambda x: np.NaN if datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ") < most_early_time else x
temp_df.iloc[:,1:2] = drop_na_df.iloc[:,1:2].applymap(f)

split_date_df = temp_df.dropna(axis=0, how='any')
split_date_df.describe()

,0,1,2,3,4,5
count,1645146,1645146,1645146,1645146,1645146,1645146
unique,902,1484806,39084,38493,339118,264073
top,user_000791,2009-02-26T21:29:15Z,164f0d73-1234-4e2c-8743-d77bf2191051,Kanye West,f06ab747-9507-4373-966b-71b797c2466a,Intro
freq,24413,193,13363,13363,1303,1346


In [2]:
file_name = 'split_date_set'
#split_date_df.to_pickle(data_dir + file_name)
split_date_df = pd.read_pickle(data_dir + file_name)

In [3]:
#Set Session_time which is the minimum time between two songs
Session_time = timedelta(minutes=20)

In [4]:
idx = 0
data = split_date_df.iloc[idx:idx+1,:].values[0]
print(data)
test_time1 = datetime.strptime('2009-05-04T13:06:09Z', "%Y-%m-%dT%H:%M:%SZ")
test_time2 = datetime.strptime('2009-05-04T13:25:09Z', "%Y-%m-%dT%H:%M:%SZ")
test_time2 - test_time1 > Session_time

['user_000001' '2009-05-04T13:06:09Z'
 'a7f7df4a-77d8-4f12-8acd-5c60c93f4de8' '坂本龍一'
 'f7c1f8f8-b935-45ed-8fc8-7def69d92a10' 'The Last Emperor (Theme)']


False

In [5]:
dataset_dic = {}
#for idx in range(split_date_df.shape[0]-1,-1,-1):
for idx in range(split_date_df.shape[0]-1,-1,-1):
    data = split_date_df.iloc[idx:idx+1,:].values[0]
    #print(data)
    if not data[0][5:] in dataset_dic.keys():
        #Session List
        dataset_dic[data[0][5:]] = []
        current_time = datetime.strptime(data[1], "%Y-%m-%dT%H:%M:%SZ")
        #Session
        dataset_dic[data[0][5:]].append([])
        #log
        dataset_dic[data[0][5:]][0].append([data[3],data[5]])
    else:
        #log time
        temp_time = datetime.strptime(data[1], "%Y-%m-%dT%H:%M:%SZ")
        if temp_time - current_time > Session_time:
            #new session
            dataset_dic[data[0][5:]].append([])
            #Put the log to the len(session_list)-1 of session_list(current_session)
            dataset_dic[data[0][5:]][len(dataset_dic[data[0][5:]])-1].append([data[3],data[5]])
            current_time = temp_time
        else:
            #Put the log to the len(session_list)-1 of session_list(current_session)
            dataset_dic[data[0][5:]][len(dataset_dic[data[0][5:]])-1].append([data[3],data[5]])
            current_time = temp_time

In [7]:
users = list(dataset_dic.values())
users = users.copy()

In [8]:
#Set the minimun number of songs in a session
Limit_of_Session = 6

In [9]:
#remove len of session_list < 6
#the first of session is the oldest
#user
for session_list in users:
    #session list
    session_list_copy = session_list.copy()
    #print(len(session_list))
    for session in session_list_copy:
        if len(session) < Limit_of_Session:
            session_list.remove(session)
    #print(len(session_list))
    
users_copy = users.copy()
for session_list in users_copy:
    if len(session_list) == 0:
        users.remove(session_list)

In [4]:
#Save Users=[session list,session list,...]=[[session,session,...],[session,session,...],...]

#print(len(users))

#Save
#with open(data_dir + 'users.pkl', 'wb') as f:
#    pickle.dump(users, f)
#load
with open(data_dir + 'users.pkl', 'rb') as f:
    users = pickle.load(f)
print(len(users))

742


In [5]:
Data_Size = 6

In [6]:
SongData = []
for session_list in users:
    for session in session_list:
        for idx in range(len(session)-Data_Size-1):
            SongData.append(session[idx:idx+Data_Size])

In [14]:
#Find the Unique Song List
unique_song = []
unique_song_name = []
for session_list in tqdm.tqdm(users):
    for session in session_list:
        for log in session:
            song_name = log[0] + log[1]
            if not song_name in unique_song_name:
                unique_song_name.append(song_name)
                unique_song.append([log[0], log[1]])

100%|████████████████████████████████████████████████████████████████████████████████| 742/742 [37:48<00:00,  3.06s/it]


In [2]:
#print(len(unique_song))
#print(len(unique_song_name))

#Save
#with open(data_dir + '1M_unique_song.pkl', 'wb') as f:
#    pickle.dump(unique_song, f)
    
#load
with open(data_dir + '1M_unique_song.pkl', 'rb') as f:
    unique_song = pickle.load(f)
print(len(unique_song))

173327


In [3]:
from LastfmTag import Search_Tag

#Use unique song List find the song which have 'Invalid Tag'
#OK_unique_song = [['Kid Sister', 'Control'],...]
#Tag_List = [['dance', 'rap', 'hip hop'],...]
#Invalid = [[Invalid_Artist, Invalid_Song],...]

OK_unique_song = []
Tag_List = []
Invalid = []

max_len = len(unique_song)

for idx in tqdm.tqdm(range(max_len)):
    artist_name = unique_song[idx][0]
    track_name = unique_song[idx][1]
    OK_unique_song, Tag_List, Invalid = Search_Tag(OK_unique_song, Tag_List, Invalid, artist_name, track_name)
    #time.sleep(0.0001)

100%|███████████████████████████████████████████████████████████████████████| 173327/173327 [77:56:05<00:00,  1.62s/it]


In [3]:
#Invalid
print('Invalid')
#print(len(Invalid))
#Save
#with open(data_dir + 'Invalid.pkl', 'wb') as f:
#    pickle.dump(Invalid, f)
#load
with open(data_dir + 'Invalid.pkl', 'rb') as f:
    Invalid = pickle.load(f)
print(len(Invalid))


#OK_unique_song
print('OK_unique_song')
#print(len(OK_unique_song))
#Save
#with open(data_dir + 'OK_unique_song.pkl', 'wb') as f:
#    pickle.dump(OK_unique_song, f)
#load
with open(data_dir + 'OK_unique_song.pkl', 'rb') as f:
    OK_unique_song = pickle.load(f)
print(len(OK_unique_song))


#Tag_List
print('Tag_List')
#print(len(Tag_List))
#Save
#with open(data_dir + 'Tag_List.pkl', 'wb') as f:
#    pickle.dump(Tag_List, f)
#load
with open(data_dir + 'Tag_List.pkl', 'rb') as f:
    Tag_List = pickle.load(f)
print(len(Tag_List))

Invalid
43318
OK_unique_song
130009
Tag_List
130009


In [8]:
OK_unique_tag = []
for song_tag in tqdm.tqdm(Tag_List):
    for tag in song_tag:
        if not tag in OK_unique_tag:
            OK_unique_tag.append(tag)
#Tag_List
print('OK_unique_tag')
#print(len(Tag_List))
#Save
with open(data_dir + 'OK_unique_tag.pkl', 'wb') as f:
    pickle.dump(OK_unique_tag, f)
#load
with open(data_dir + 'OK_unique_tag.pkl', 'rb') as f:
    OK_unique_tag = pickle.load(f)
print(len(OK_unique_tag))

100%|██████████| 130009/130009 [00:04<00:00, 27429.44it/s]

OK_unique_tag
16141


In [9]:
#Remove the unique_song's song which in the Invalid lsit to check the length
#Delete invalid song which is in SongData
#SongData剔除包含'不符合的song, Asrtist'的Data
OK_Song_Data = []
for data in tqdm.tqdm(SongData):
    Invalid_log = False
    for log in data:
        artist = log[0]
        track = log[1]
        if ([artist, track] in Invalid) or not([artist, track] in OK_unique_song):
            Invalid_log = True
            break
    if not Invalid_log:
        OK_Song_Data.append(data)
print(len(OK_Song_Data))

100%|██████████| 1119883/1119883 [2:27:14<00:00, 126.76it/s] 

457051


In [10]:
print(len(SongData))
print(len(OK_Song_Data))

1119883
457051


In [2]:
#OK_Song_Data
print('OK_Song_Data')
#print(len(Tag_List))
#Save
#with open(data_dir + 'OK_Song_Data.pkl', 'wb') as f:
#    pickle.dump(OK_Song_Data, f)
#load
with open(data_dir + 'OK_Song_Data.pkl', 'rb') as f:
    OK_Song_Data = pickle.load(f)
print(len(OK_Song_Data))

OK_Song_Data
457051


In [4]:
#OK_unique_song, Tag_List
OK_Tag_Data = []
for data in tqdm.tqdm(OK_Song_Data):
    OK_Tag_Data.append([])
    for log in data:
        artist = log[0]
        track = log[1]
        #print([artist, track])
        tag_idx = OK_unique_song.index([artist, track])
        #print(tag_idx)
        OK_Tag_Data[len(OK_Tag_Data) - 1].append(Tag_List[tag_idx])
        #print(Tag_List[tag_idx])
        #break
    #break
print(len(OK_Tag_Data))

100%|██████████| 457051/457051 [42:27<00:00, 179.41it/s] 

457051


In [5]:
#OK_Tag_Data
print('OK_Tag_Data')
#print(len(Tag_List))
#Save
#with open(data_dir + 'OK_Tag_Data.pkl', 'wb') as f:
#    pickle.dump(OK_Tag_Data, f)
#load
with open(data_dir + 'OK_Tag_Data.pkl', 'rb') as f:
    OK_Tag_Data = pickle.load(f)
print(len(OK_Tag_Data))

OK_Tag_Data
457051
